In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.parent.absolute()))
import config
from context import Context
from contracts.farm_contract import FarmContract
from contracts.pair_contract import PairContract
from contracts.position_creator_contract import PositionCreatorContract
from contracts.simple_lock_energy_contract import SimpleLockEnergyContract
from utils.utils_generic import log_warning, log_step_pass, log_step_fail

context = Context()

EGLDMEX_POOL_ADDRESS = "erd1qqqqqqqqqqqqqpgqa0fsfshnff4n76jhcye6k7uvd7qacsq42jpsp6shh2"
EGLDFOXSY_POOL_ADDRESS = "erd1qqqqqqqqqqqqqpgq5jnjpsukhl295ry3wjrd3gtff0amrgux2jpsz3reum"

mex_pair_contract = PairContract.load_contract_by_address(EGLDMEX_POOL_ADDRESS)
foxsy_pair_contract = PairContract.load_contract_by_address(EGLDFOXSY_POOL_ADDRESS)
log_warning(f'Pair contract: {mex_pair_contract.lpToken} {mex_pair_contract.address}')
log_warning(f'Pair contract: {foxsy_pair_contract.lpToken} {foxsy_pair_contract.address}')

energy_factory: SimpleLockEnergyContract
energy_factory = context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0]

mex_farm_contract = FarmContract.load_contract_by_address("erd1qqqqqqqqqqqqqpgqtdc0rzjwmp9qs0h0wj9pjwxnp5vjc9hdkp2swd4dxz")
foxsy_farm_contract = FarmContract.load_contract_by_address("erd1qqqqqqqqqqqqqpgqu288dj3qgrelujal5wynsv5zrf7u0hwlkp2se383n7")
log_warning(f'New MEX Farm contract: {mex_farm_contract.farmToken}')
log_warning(f'New FOXSY Farm contract: {foxsy_farm_contract.farmToken}')

position_creator_contract: PositionCreatorContract
position_creator_contract = context.get_contracts(config.POSITION_CREATOR)[0]

In [ ]:
from contracts.metastaking_contract import MetaStakingContract
from contracts.staking_contract import StakingContract

metastaking_contract = MetaStakingContract.load_contract_by_address("erd1qqqqqqqqqqqqqpgqqcgdee7ut4qmx942fpgtk9da53acmserkp2sez9f70")
staking_contract = StakingContract.load_contract_by_address("erd1qqqqqqqqqqqqqpgq2e4fmlhcv90wrxl2acjpcu3g2l9rpkttx9rs8pppzp")
log_warning(f'Metastake contract: {metastaking_contract.metastake_token}')
log_warning(f'Stake contract: {staking_contract.farm_token}')

Change owners

In [ ]:
new_owner = "erd1ss6u80ruas2phpmr82r42xnkd6rxy40g9jl69frppl4qez9w2jpsqj8x97"

In [ ]:
mex_farm_contract.change_owner_address(context.deployer_account, context.network_provider.proxy, new_owner)

In [ ]:
foxsy_farm_contract.change_owner_address(context.deployer_account, context.network_provider.proxy, new_owner)

In [ ]:
metastaking_contract.change_owner_address(context.deployer_account, context.network_provider.proxy, new_owner)

Take ownership

In [ ]:
old_owner = "erd19uysnnzwnzm3klq8qzv7zprw2d3yj38hx0ac9cuqvl0j2jr5kp2sflvmyv"

In [ ]:
mex_farm_contract.update_owner_or_admin(context.deployer_account, context.network_provider.proxy, old_owner)

In [ ]:
foxsy_farm_contract.update_owner_or_admin(context.deployer_account, context.network_provider.proxy, old_owner)

EGLDMEX farm config

In [ ]:
mex_pair_contract.whitelist_contract(context.deployer_account, context.network_provider.proxy, mex_farm_contract.address)

In [ ]:
energy_factory.add_sc_to_whitelist(context.deployer_account, context.network_provider.proxy, mex_farm_contract.address)

In [ ]:
energy_factory.set_transfer_role_locked_token(context.deployer_account, context.network_provider.proxy, [mex_farm_contract.address])

EGLDFOXSY farm config

In [ ]:
foxsy_pair_contract.whitelist_contract(context.deployer_account, context.network_provider.proxy, foxsy_farm_contract.address)

In [ ]:
energy_factory.add_sc_to_whitelist(context.deployer_account, context.network_provider.proxy, foxsy_farm_contract.address)

In [ ]:
energy_factory.set_transfer_role_locked_token(context.deployer_account, context.network_provider.proxy, [foxsy_farm_contract.address])

FOXSY Metastaking config

In [ ]:
foxsy_pair_contract.whitelist_contract(context.deployer_account, context.network_provider.proxy, metastaking_contract.address)

In [ ]:
staking_contract.whitelist_contract(context.deployer_account, context.network_provider.proxy, metastaking_contract.address)

In [ ]:
energy_factory.set_transfer_role_locked_token(context.deployer_account, context.network_provider.proxy, [metastaking_contract.address])

Position creator whitelist

In [ ]:
mex_farm_contract.add_contract_to_whitelist(context.deployer_account, context.network_provider.proxy, position_creator_contract.address)

In [ ]:
foxsy_farm_contract.add_contract_to_whitelist(context.deployer_account, context.network_provider.proxy, position_creator_contract.address)

In [ ]:
metastaking_contract.whitelist_contract(context.deployer_account, context.network_provider.proxy, position_creator_contract.address)

Set farm rewards per block for farms

In [ ]:
mex_farm_contract.set_rewards_per_block(context.deployer_account, context.network_provider.proxy, 61100*10**18)

In [ ]:
old_mex_farm_contract = FarmContract.load_contract_by_address("erd1qqqqqqqqqqqqqpgqapxdp9gjxtg60mjwhle3n6h88zch9e7kkp2s8aqhkg")
log_warning(f'Old MEX Farm contract: {old_mex_farm_contract.farmToken}')
old_mex_farm_contract.set_rewards_per_block(context.deployer_account, context.network_provider.proxy, 61100*10**18)

Start new farms

In [ ]:
mex_farm_contract.resume(context.deployer_account, context.network_provider.proxy)

Set rewards active for new farms

In [ ]:
mex_farm_contract.start_produce_rewards(context.deployer_account, context.network_provider.proxy)

WXMEX whitelist

In [ ]:
from contracts.lk_wrap_contract import LkWrapContract

lk_wrap_contract: LkWrapContract
lk_wrap_contract = context.get_contracts(config.LK_WRAPS)[0]

lk_wrap_contract.set_transfer_role_wrapped_token(context.deployer_account, context.network_provider.proxy, ["erd1qqqqqqqqqqqqqpgq9slqavjm7pglxgzuskwlvnq53gnk02vndfysq95mpq"])